In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [254]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def test_da_shit(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

def mean_encoder(df, col, target = 'Sales'):
    Mean_encoded_subject = df.groupby([col])[target].mean().to_dict() 
    return df[col].map(Mean_encoded_subject)

## Dirty Data Imports (don't run)

In [123]:
# train = pd.read_csv('data/train.csv', parse_dates = True)
# stores = pd.read_csv('data/store.csv')

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Clean Data imports (run these!)

In [ ]:
no_null_sales = pd.read_csv('data/Super_clean_data.csv', parse_dates = True)
stores = pd.read_csv('data/stores_light.csv')

## Base cleaning (don't run)

In [125]:
# fillna_storetype_a = stores.groupby('StoreType').median().loc['a', 'CompetitionDistance']
# fillna_storetype_b = stores.groupby('StoreType').median().loc['b', 'CompetitionDistance']
# fillna_storetype_c = stores.groupby('StoreType').median().loc['c', 'CompetitionDistance']
# fillna_storetype_d = stores.groupby('StoreType').median().loc['d', 'CompetitionDistance']
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'a')] = fillna_storetype_a
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'b')] = fillna_storetype_b
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'c')] = fillna_storetype_c
# stores.loc[(stores['CompetitionDistance'].isnull()) & (stores.loc[:,'StoreType'] == 'd')] = fillna_storetype_d

# train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Open'] == 0), 'Sales'] = 0
# train.loc[(train.loc[:, 'Sales'].isnull()) & (train['Customers'] == 0), 'Sales'] = 0

# train.loc[(train.loc[:, 'Customers'].isnull()) & (train['Sales'] == 0), 'Customers'] = 0

# no_null_sales = train.loc[(train['Sales'].notnull()) | (train['Open'] == 0), :]
# no_null_sales.loc[:,'Sales'].fillna(0, inplace=True)

# no_null_sales = no_null_sales.loc[no_null_sales['Store'].notnull(), :]

# no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
# no_null_sales.loc[:,'DayOfWeek'] = no_null_sales.loc[:,'Date'].dt.weekday + 1

# no_null_sales.loc[no_null_sales['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
# no_null_sales.loc[no_null_sales['StateHoliday'] == 0, 'StateHoliday'] = '0'

# no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] == 0),'Open'] = 0
# no_null_sales.loc[(no_null_sales['Open'].isnull()) & (no_null_sales['Sales'] > 0),'Open'] = 1

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Baseline 1

In [126]:
X = no_null_sales[['Store']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = 'Store').loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

40.58064127185794

## Baseline 2

In [127]:
X = no_null_sales[['Store', 'DayOfWeek']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek']).loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

25.42650037519698

## Baseline 3

In [213]:
X = no_null_sales[['Store', 'DayOfWeek', 'Open']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek', 'Open']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek', 'Open']).loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

23.438336545019375

## Further cleaning (don't run unless you're sure!)

In [129]:
#WILL TAKE 30 MINS TO RUN!!!!!!
#Slow and ugly way to fill in NaNs with the mode
holiday_nulls = no_null_sales.loc[no_null_sales['StateHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'StateHoliday'].mode()[0]
    no_null_sales.loc[i, 'StateHoliday'] = mode
    
holiday_nulls = no_null_sales.loc[no_null_sales['SchoolHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = no_null_sales.loc[no_null_sales['Date'] == date, 'SchoolHoliday'].mode()[0]
    no_null_sales.loc[i, 'SchoolHoliday'] = mode

In [148]:
customer_means = no_null_sales.groupby(['Store', 'DayOfWeek']).mean()['Customers']

customer_nulls = no_null_sales.loc[no_null_sales['Customers'].isnull()]
for i, row in customer_nulls.iterrows():
    store = row['Store']
    DayOfWeek = row['DayOfWeek']
    mean = customer_means.loc[store, DayOfWeek]
    no_null_sales.loc[i, 'Customers'] = mean

In [ ]:
no_null_sales.reset_index(inplace = True, drop = True)

null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    above_and_below = no_null_sales.loc[i-1, 'Promo'] + no_null_sales.loc[i+1, 'Promo']
    if above_and_below == 2:
        no_null_sales.loc[i, 'Promo'] = 1
    elif above_and_below == 0:
        no_null_sales.loc[i, 'Promo'] = 0
        
null_promo = no_null_sales.loc[no_null_sales['Promo'].isnull()]

for i, row in null_promo.iterrows():
    date = str(row['Date'])
    mean = no_null_sales.groupby('Date').mean().loc[date, 'Promo']
    if mean > 0.9:
        no_null_sales.loc[i, 'Promo'] = 1
    else:
        no_null_sales.loc[i, 'Promo'] = 0

## Random Forest 1st attempt

In [87]:
rf_set = no_null_sales.drop(['Date', 'Customers', 'SchoolHoliday', 'Promo'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

23.334534802625598

In [192]:
no_null_sales.drop('index', axis = 1, inplace = True)

## 20 % sample experiments

In [245]:
sample = no_null_sales.sample(frac = 0.2, random_state = 42)

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
559764,2014-06-15,798.0,7,0.0,0.0,0.0,0.0,0,0.0
122249,2013-04-26,739.0,5,7899.0,654.0,1.0,1.0,0,0.0
297756,2013-10-10,225.0,4,5662.0,609.0,1.0,1.0,0,0.0
239136,2013-08-15,592.0,4,5365.0,707.0,1.0,1.0,0,1.0
48008,2013-02-15,930.0,5,5002.0,893.0,1.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
245313,2013-08-21,1092.0,3,9339.0,963.0,1.0,0.0,0,1.0
289296,2013-10-02,43.0,3,6338.0,625.0,1.0,0.0,0,0.0
546435,2014-06-03,101.0,2,8613.0,813.0,1.0,1.0,0,0.0
25668,2013-01-25,23.0,5,6618.0,572.0,1.0,1.0,0,0.0


In [246]:
# Without promo

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Promo'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

24.817183927889534

In [247]:
#With Promo

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.361161517312457

In [248]:
##With School holiday

rf_set = sample.drop(['Date', 'Customers'], axis = 1)
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.715715824411912

In [251]:
sample['Month'] = sample['Date'].dt.month
sample['Week'] = sample['Date'].dt.week

In [261]:
#With Month (tried both with and without mean encoder, neither was good)
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Week'], axis = 1)
rf_set['Month'] = mean_encoder(rf_set, 'Month')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

21.661258781878868

In [267]:
#With Week (tried both with and without mean encoder, neither was good)
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month'], axis = 1)
rf_set['Week'] = mean_encoder(rf_set, 'Week')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

23.50369032548846

In [265]:
rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Week'], axis = 1)
rf_set.loc[rf_set['Month'] != 12, 'Month'] = 0
rf_set.loc[rf_set['Month'] == 12, 'Month'] = 1
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.757512325539558

In [269]:
#Store mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['Store'] = mean_encoder(rf_set, 'Store')
rf_set.loc[:,'StateHoliday'] = rf_set.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.36365573605935

In [271]:
#State-Holiday mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.35301377936429

In [318]:
#Weekday mean-encoded

rf_set = sample.drop(['Date', 'Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
rf_set['DayOfWeek'] = mean_encoder(rf_set, 'DayOfWeek')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.365006672638962

In [329]:
# # DON'T RUN!!
## Making dayBeforeHoliday feature, didn't help 

# aDay = datetime.timedelta(days = 1)

# sample2 = no_null_sales.copy()
# sample2.loc[:,'StateHoliday'] = sample2.loc[:,'StateHoliday'].apply(lambda x: 0 if x=='0' else 1)
# mapping = sample2.groupby(['Date']).mean()['StateHoliday']
# sample2.loc[:,'DayBeforeHoliday'] = 0
# for i, row in sample2.iterrows():
#     if i == 0:
#         print('Hi!')
#     else:
#         date1 = str(row['Date'])
#         date2 = str(row['Date'] + aDay)
#         if mapping.loc[date2] > 0.5 and mapping.loc[date1] == 0:
#             sample2.loc[i, 'DayBeforeHoliday'] = 1

Hi!


KeyError: '2014-08-01 00:00:00'

In [339]:
#With DayBeforeHoliday

with_dbh = sample2.sample(frac = 0.2, random_state = 42)

rf_set = with_dbh.drop(['Date', 'Customers', 'SchoolHoliday'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

19.56220140476341

In [343]:
#Date mean-encoded

rf_set = sample.drop(['Customers', 'SchoolHoliday', 'Month', 'Week'], axis = 1)
rf_set['StateHoliday'] = mean_encoder(rf_set, 'StateHoliday')
rf_set['Date'] = mean_encoder(rf_set, 'Date')
X = rf_set.drop('Sales', axis = 1)
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

21.926008486479244